In [19]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

WCZYTANIE DANYCH

In [20]:
csv_data = pd.read_csv('healthcare-dataset-stroke-data.csv')
csv_data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [21]:
csv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [22]:
csv_data.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [23]:
csv_data.shape

(5110, 12)

WYBIERAMY CECHY I TARGETY

In [24]:
features = ['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi']
target = 'stroke'

TWORZENIE ZBIORU CECH I ZBIORU TERGET

In [25]:
X = csv_data.loc[:, features]
y = csv_data.loc[:, target]

UZUPEŁNIAMY BRAKUJĄCE WARTOŚCI KOLUMNY BMI

In [26]:
X['bmi'].fillna(X['bmi'].mean(), inplace=True)

In [27]:
X.isnull().sum()

age                  0
hypertension         0
heart_disease        0
avg_glucose_level    0
bmi                  0
dtype: int64

SHUFFLE

In [28]:
X, y = shuffle(X, y, random_state=42)

PODZIAŁ NA DANE TRENINGOWE 80%, WALIDACYJNE 10% I DO TESTOWANIA 10%

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

NORMALIZACJA DANYCH WEJŚCIOWYCH

In [30]:
X_train = tf.keras.utils.normalize(X_train.values, axis=1)
X_val = tf.keras.utils.normalize(X_val.values, axis=1)
X_test = tf.keras.utils.normalize(X_test.values, axis=1)

BUDOWA MODELU

In [31]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

KOMPILACJA MODELU

In [32]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

TRENOWANIE MODELU

In [33]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3)

model.fit(X_train, y_train, epochs=20, callbacks=[early_stopping], validation_data=(X_val, y_val), verbose=1)

Epoch 1/20
115/115 [==============================] - 1s 3ms/step - loss: 0.2391 - accuracy: 0.9511 - val_loss: 0.1768 - val_accuracy: 0.9560
Epoch 2/20
115/115 [==============================] - 0s 1ms/step - loss: 0.1870 - accuracy: 0.9511 - val_loss: 0.1728 - val_accuracy: 0.9560
Epoch 3/20
115/115 [==============================] - 0s 2ms/step - loss: 0.1816 - accuracy: 0.9511 - val_loss: 0.1683 - val_accuracy: 0.9560
Epoch 4/20
115/115 [==============================] - 0s 1ms/step - loss: 0.1789 - accuracy: 0.9511 - val_loss: 0.1620 - val_accuracy: 0.9560
Epoch 5/20
115/115 [==============================] - 0s 2ms/step - loss: 0.1765 - accuracy: 0.9511 - val_loss: 0.1621 - val_accuracy: 0.9560
Epoch 6/20
115/115 [==============================] - 0s 1ms/step - loss: 0.1759 - accuracy: 0.9511 - val_loss: 0.1573 - val_accuracy: 0.9560
Epoch 7/20
115/115 [==============================] - 0s 1ms/step - loss: 0.1738 - accuracy: 0.9511 - val_loss: 0.1562 - val_accuracy: 0.9560
Epoch 

PRZEWIDYWANIE

In [34]:
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

32/32 [==============================] - 0s 1ms/step


DOKŁADNOŚĆ MODELU

In [35]:
accuracy = accuracy_score(y_test, y_pred)
print('Dokładność modelu:', accuracy)

Dokładność modelu: 0.9500978473581213


PORÓWNANIE

In [36]:
df_comparison = pd.DataFrame({'Dane aktualne': y_test.values.flatten(), 'Dane przewidywane': y_pred.flatten()})
print(df_comparison)

      Dane aktualne  Dane przewidywane
0                 0                  0
1                 0                  0
2                 0                  0
3                 1                  0
4                 0                  0
...             ...                ...
1017              0                  0
1018              0                  0
1019              0                  0
1020              0                  0
1021              0                  0

[1022 rows x 2 columns]
